# Types and Dispatch in Julia

Julia is built around types.

Software architectures in Julia are built around good use of the type system.

# Abstract vs concrete types

*Concrete types* are the types of objects. They specify the data structure of an object.

*Abstract types* cannot be instantiated. They define sets of related concrete types (their descendants) by their behavior.

In [1]:
typeof(3)

Int64

In [2]:
typeof(3.0)

Float64

In [3]:
isconcretetype(Float64)

true

In [4]:
isabstracttype(Number)

true

In [5]:
isabstracttype(Real)

true

### Duck typing

A `Number` is some abstract type that can do things like `+`,`-`,`*`, and `/`. In this category we have (concrete) things like `Float64` and `Int32`.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

### Inspecting the type tree

In [6]:
supertype(Float64)

AbstractFloat

In [7]:
supertype(AbstractFloat)

Real

In [8]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16
 Float32
 Float64

In [9]:
supertype(Real)

Number

In [11]:
supertype(Number)

Any

Everything is a subtype of `Any`

In [12]:
Number <: Any

true

In [13]:
Float64 <: Any

true

In [14]:
Int32 <: Any

true

In [15]:
Int32 <: String

false

There is also `isa` for objects:

In [16]:
3.0 isa Float64

true

In [17]:
3 isa Float64

false

In [18]:
typeof(3) <: Float64

false

We define a function that, given a concrete type `T`, prints the single branch of the type tree that leads from the top node `Any` to the leave `T`.

In [30]:
show_supertypes(T) = print(join(supertypes(T), " <: "))

show_supertypes (generic function with 1 method)

In [31]:
show_supertypes(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any

In [32]:
show_supertypes(String)

String <: AbstractString <: Any

Let's extract a bunch of branches

In [22]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

┌ Info: Precompiling AbstractTrees [1520ce14-60c1-5f80-bbc7-55ef81b5835c]
└ @ Base loading.jl:1278


In [23]:
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


Note that **concrete types are the leaves of the type tree**.

Abstract types are nodes in the type graph.

# Functions, Methods, and Dispatch

Let's define a *function* that calculates the absolute value of a number (like Julias `abs` already does).

How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?

Presumably:
* Real number: "Drop the sign." => `myabs(-4.32) = 4.32`
* Complex number: "Square root of z times the complex conjugate of z." => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [ ]:
myabs(x::Float64) = sign(x) * x

In [ ]:
myabs(-4.32)

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
myabsthatdoesntexist(1.0 + 1.0im)

In [ ]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
methods(myabs)

One can check which particular method is being used through the `@which` macro.

In [ ]:
@which myabs(-4.32)

In [ ]:
@which myabs(1.0 + 1.0im)

Note that we should better loosen our type restrictions:

In [ ]:
myabs(-3)

In [ ]:
myabs(1 + 1im)

In [ ]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

In [ ]:
myabs(-3)

# Multiple Dispatch

Julia's dispatch mechanism always chooses the *most specific method* for the given input types.

In [1]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [2]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[1]:5
[2] f(a::Number, b::Number) in Main at In[1]:2
[3] f(a::Number, b) in Main at In[1]:3
[4] f(a, b::Number) in Main at In[1]:4
[5] f(a, b) in Main at In[1]:1

In [3]:
f(1.5, 2)

"a and b are both numbers"

In [4]:
f(1, "NRW!")

"a is a number"

In [5]:
f(1, 2)

"a and b are both integers"

In [6]:
f("Hello", "World!")

"fallback"

In [7]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[1]:5

In [8]:
@which f(1, "NRW!")

f(a::Number, b) in Main at In[1]:3

In [9]:
methods(+)

# 184 methods for generic function "+":
[1] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue) in Pkg.Resolve at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Pkg/src/Resolve/fieldvalues.jl:43
[2] +(x::Float32, y::Float32) in Base at float.jl:400
[3] +(a::Float16, b::Float16) in Base at float.jl:398
[4] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[5] +(z::Complex{Bool}, x::Real) in Base at complex.jl:301
[6] +(::Missing, ::Missing) in Base at missing.jl:114
[7] +(::Missing) in Base at missing.jl:100
[8] +(::Missing, ::Number) in Base at missing.jl:115
[9] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight) in Pkg.Resolve at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Pkg/src/Resolve/versionweights.jl:22
[10] +(x::Float64, y::Float64) in Base at float.jl:401
[11] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[12] +(x::Bool, y::Bool) in Base at bool.jl:96
[13] +(x::Bool) in Base at bool.jl:93
[14] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:103
[15] +(x::Bool, z::Complex) in Base at complex.jl:293
[16] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/periods.jl:347
[17] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/periods.jl:345
[18] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/periods.jl:375
[19] +(x::Dates.Date, y::Dates.Day) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:74
[20] +(x::Dates.Date, y::Dates.Week) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:72
[21] +(dt::Dates.Date, z::Dates.Month) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:54
[22] +(dt::Dates.Date, y::Dates.Year) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:27
[23] +(dt::Dates.Date, t::Dates.Time) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:19
[24] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:116
[25] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:399
[26] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:518
[27] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:517
[28] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:516
[29] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:476
[30] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:524
[31] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:530
[32] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:548
[33] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:541
[34] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:535
[35] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:394
[36] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:363
[37] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:370
[38] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:378
[39] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:386
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:47
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Applications/Julia-1.5.app/Contents/Resources/julia/share/julia/stdlib/v1.5/Dates/src/arithmetic.jl:23
[42] +(x::Dates.DateTime, y::Dates.Period) in D

In [10]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:96

In [11]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:251

Julia's standard functions are not special by any means.

We can easily modify or add methods to them as well.

In [12]:
import Base: + # we have to import functions to override/extend them
+(x::String, y::String) = x * " " * y

+ (generic function with 185 methods)

In [16]:
"Hello" + "NRW!"

"Hello NRW!"

(**Important note**: as we neither own the `+` function nor the `String` type the above is **type piracy** and should generally be avoided.)

Any function based on the `+` operation can now handle `String`s as well.

In [14]:
sum(["This", "works", "although", "we", "never", "touched", "sum!"])

"This works although we never touched sum!"

It happens rarely, but it can happen that there is no unique most specific method:

In [ ]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

# Parametric types

Types can have *type parameters*. The most prominent example is Julia's array type.

In [ ]:
M = rand(2,2)

In [ ]:
typeof(M)

Here, `Array` is a parametric array datatype. Its type parameters `Float64` and `2` indicate the type of the element the array can hold and and its dimensionality. Hence, we have a matrix of floating point numbers.

This generalizes as expected. Here, a matrix of `String`s:

In [ ]:
M = fill("Oulu", 2,2)

In [ ]:
eltype(M)

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [ ]:
v = [rand(2,2) for i in 1:3]

In [ ]:
eltype(v)

Since vectors and matrices pop up so frequently, Julia has a nice shortcut type alias for them.

In [ ]:
Vector{Float64} === Array{Float64, 1}

In [ ]:
Matrix{Float64} === Array{Float64, 2}

Another example of a parametric type is the `Tuple`.

In [ ]:
(1,2.0,"3")

In [ ]:
typeof((1,2.0,"3"))

### `UnionAll` types and `where`

Note that parametric types have the following (somewhat counterintuitive at first) property

In [ ]:
Vector{Float64} <: Vector{Real}

although we have

In [ ]:
Float64 <: Real

(In parlance of type theory, Julia's type parameters are *invariant*. Read [this](https://discourse.julialang.org/t/problem-with-complex-rationals/9474/7) for a simple explanation why this property is useful/necessary for performance. See also the [parametric type section](https://docs.julialang.org/en/v1/manual/types/#Parametric-Composite-Types) of the documentation.)

How can we understand this property? `Vector{Real}` is a concrete container type - it describes a vector of values that individually have a type `T <: Real`  - and concrete types don't have subtypes.

In [ ]:
isconcretetype(Vector{Real})

In [ ]:
Real[1, 2.2, 13f0]

What we often actually *mean* is

In [ ]:
Vector{Float64} <: Vector{T} where T<:Real

Here, `Vector{T} where T <: Real` describes the *set* of concrete `Vector` types whose elements are of any specific single type `T` that is a subtype of `Real`.

In [ ]:
Vector{Int64} <: Vector{T} where T<:Real

In [ ]:
Vector{Real} <: Vector{T} where T<:Real

Using this notation, our `Vector{Real}` from above corresponds to `Vector{T where T<:Real}`

In [ ]:
Vector{Real} === Vector{T where T<:Real}

### Type parameters in function signatures

In [ ]:
h(x::Integer) = typeof(x)

In [ ]:
h(x::T) where T = T

**Quick exercise**: Write a single-argument function that takes *any real matrix* as input and, for example, returns the element type of the matrix.

**Solution:**
<details>
  <summary>Click to reveal</summary>
<br>
    
```julia
g(x::Matrix{T}) where T<:Real = T
```

or alternatively

```julia
g(x::Matrix{<:Real}) = eltype(x)
```
</details>

**Test:**

In [ ]:
g(rand(Float32,2,2))

In [ ]:
g(rand(Int16,2,2))

# "Diagonal" dispatch

In [ ]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

In [ ]:
d(3, 4)

In [ ]:
d(3.0, 1.0)

In [ ]:
d(1, 4.2)

# Duck typing examples

### `UnitRange`

In [ ]:
x = 1:30

In [ ]:
typeof(x)

In [ ]:
typeof(x) <: AbstractArray

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [ ]:
x[3]

In [ ]:
size(x)

In [ ]:
eltype(x)

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [ ]:
fieldnames(typeof(x))

or just by inspecting the source code

In [ ]:
@which UnitRange(1, 30)

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [ ]:
@time collect(1:10000000);

But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [ ]:
@time 1:10000000;

Yet, in code they *act* the same way.

# Other types

* Union types: `Union{Float64, Int32}`
* [Bitstypes](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#man-bits-types-1) (check with `isbits(x)`, `isbitstype(T)`)
* [Value types](https://docs.julialang.org/en/v1/manual/types/#%22Value-types%22-1) (allows dispatch on values)

See https://docs.julialang.org/en/latest/manual/types/ for more.

# Extra: slurping and splatting

In [ ]:
f(x...) = println(x) # slurping

In [ ]:
f(3, 1.2, "Oulu")

In [ ]:
g(x::Vector) = +(x...) # splat vector into addition operation

In [ ]:
g([1,2,3])

# Core messages of this Notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.